# Advent of Code

## 2018-012-017
## 2018 017

https://adventofcode.com/2018/day/17

In [5]:
import re

import numpy as np



# Parse the input

input_path = 'input.txt'
def parse_input_fixed(input_path):

    clay_positions = []

    with open(input_path, 'r') as file:

        for line in file:

            line = line.strip()

            if line.startswith("x"):

                x, y1, y2 = map(int, re.findall(r"x=(\d+), y=(\d+)\.\.(\d+)", line)[0])

                clay_positions.extend((x, y) for y in range(y1, y2 + 1))

            elif line.startswith("y"):

                y, x1, x2 = map(int, re.findall(r"y=(\d+), x=(\d+)\.\.(\d+)", line)[0])

                clay_positions.extend((x, y) for x in range(x1, x2 + 1))

    return clay_positions



# Retry parsing with corrected logic

clay_positions = parse_input_fixed(input_path)



# Define grid boundaries

min_x = min(x for x, y in clay_positions)

max_x = max(x for x, y in clay_positions)

min_y = min(y for x, y in clay_positions)

max_y = max(y for x, y in clay_positions)



# Extend the grid for overflow

padding = 1

grid_width = max_x - min_x + 1 + 2 * padding

grid_height = max_y + 1



# Create the grid

grid = np.full((grid_height, grid_width), '.', dtype=str)



# Place clay on the grid

for x, y in clay_positions:

    grid[y, x - min_x + padding] = '#'



# Place the water spring

spring_x = 500 - min_x + padding

grid[0, spring_x] = '+'
def simulate_water(grid, spring_x, min_y, max_y):
    flowing_water = set()
    resting_water = set()
    
    def flow_down(x, y):
        while y < max_y and grid[y, x] == '.':
            grid[y, x] = '|'
            flowing_water.add((x, y))
            y += 1
        if y >= max_y:
            return
        if grid[y, x] in ['#', '~']:
            y -= 1
            flow_out(x, y)

    def flow_out(x, y):
        left_bound = x
        while left_bound > 0 and grid[y, left_bound] == '.' and grid[y + 1, left_bound] in ['#', '~']:
            grid[y, left_bound] = '|'
            flowing_water.add((left_bound, y))
            left_bound -= 1
        right_bound = x
        while right_bound < grid.shape[1] - 1 and grid[y, right_bound] == '.' and grid[y + 1, right_bound] in ['#', '~']:
            grid[y, right_bound] = '|'
            flowing_water.add((right_bound, y))
            right_bound += 1

        if grid[y + 1, left_bound] not in ['#', '~']:
            flow_down(left_bound, y + 1)
        if grid[y + 1, right_bound] not in ['#', '~']:
            flow_down(right_bound, y + 1)

        if grid[y, left_bound] == '#' and grid[y, right_bound] == '#':
            for i in range(left_bound + 1, right_bound):
                grid[y, i] = '~'
                resting_water.add((i, y))

    flow_down(spring_x, 1)

    return flowing_water, resting_water

# Retry the simulation
flowing_water, resting_water = simulate_water(grid, spring_x, min_y, max_y)

# Count reachable tiles
reachable_tiles = len(flowing_water | resting_water)

reachable_tiles

23

In [1]:
import re
from collections import deque

# Read the input from input.txt
with open('input.txt', 'r') as file:
    input_data = file.read()

clay_coords = set()
min_x = min_y = float('inf')
max_x = max_y = float('-inf')

# Parse each line of the input
for line in input_data.strip().split('\n'):
    nums = list(map(int, re.findall(r'\d+', line)))
    if line.startswith('x'):
        x = nums[0]
        y1, y2 = nums[1], nums[2]
        for y in range(y1, y2 + 1):
            clay_coords.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)
    else:
        y = nums[0]
        x1, x2 = nums[1], nums[2]
        for x in range(x1, x2 + 1):
            clay_coords.add((x, y))
            min_x = min(min_x, x)
            max_x = max(max_x, x)
            min_y = min(min_y, y)
            max_y = max(max_y, y)

# Expand the grid boundaries a bit to account for water spreading
min_x -= 1
max_x += 1

# Initialize the grid
grid = {}
for y in range(0, max_y + 2):
    for x in range(min_x, max_x + 1):
        grid[(x, y)] = '.'

# Place the clay
for (x, y) in clay_coords:
    grid[(x, y)] = '#'

# Place the water spring
spring_x, spring_y = 500, 0
grid[(spring_x, spring_y)] = '+'

# Simulate water flow
def flow(x, y, from_above=False):
    if y > max_y:
        return
    if grid[(x, y)] in ('#', '~'):
        return
    if grid[(x, y)] == '|':
        if from_above:
            return
    grid[(x, y)] = '|'
    below = grid.get((x, y + 1), '.')
    if below == '.':
        flow(x, y + 1)
    elif below in ('#', '~'):
        # Try to spread left and right
        left_bound = x
        right_bound = x

        # Spread left
        while True:
            left = left_bound - 1
            below_left = grid.get((left, y + 1), '.')
            if grid.get((left, y), '.') in ('#', '~'):
                break
            if below_left not in ('#', '~'):
                flow(left, y)
                break
            left_bound -= 1
            grid[(left, y)] = '|'

        # Spread right
        while True:
            right = right_bound + 1
            below_right = grid.get((right, y + 1), '.')
            if grid.get((right, y), '.') in ('#', '~'):
                break
            if below_right not in ('#', '~'):
                flow(right, y)
                break
            right_bound += 1
            grid[(right, y)] = '|'

        # Check if water is settled
        is_settled = True
        for x_fill in range(left_bound, right_bound + 1):
            if grid.get((x_fill, y + 1), '.') not in ('#', '~'):
                is_settled = False
                break
        if is_settled:
            for x_fill in range(left_bound, right_bound + 1):
                grid[(x_fill, y)] = '~'
            flow(x, y - 1, from_above=True)
    else:
        # Flowing into abyss
        return

# Start the water flow
flow(spring_x, spring_y + 1)

# Count the water tiles
water_tiles = 0
for (x, y), value in grid.items():
    if min_y <= y <= max_y and value in ('|', '~'):
        water_tiles += 1

print(f"Total number of tiles the water can reach: {water_tiles}")

Total number of tiles the water can reach: 38
